In [780]:
import requests
import pandas as pd
import os.path
from datetime import datetime, timedelta, timezone
from pytz import timezone

In [781]:
def fetch():
    molde = "lat=62.73752&lon=7.15912"
    url = f"https://api.met.no/weatherapi/locationforecast/2.0/complete?{molde}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    return response.json()

In [782]:
def oslo_time():
    oslo_tid = datetime.now().astimezone(timezone("Europe/Oslo"))
    oslo_tid = (oslo_tid - timedelta(hours=1)).strftime("%Y-%m-%dT%H:00:00Z")
    return oslo_tid

In [783]:
def prepare_data(data: dict):
    # Try to handle timezone difference on hosted server
    now = oslo_time()
    data = data["properties"]["timeseries"]

    # Find the forecast we wish to log
    for entry in data:
        if entry["time"] == now:
            data = entry["data"]["instant"]["details"]
            date, time = now.replace("Z", "").split("T")
            data["time"] = time
            data["date"] = date

    df = pd.DataFrame([data])

    # Change order of columns, setting time and date first.
    cols = df.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    df = df[cols]

    return df

In [784]:
def append_to_csv(data: pd.DataFrame, path: str):
    # Only write header if the file does not already exist
    header = data.columns if not os.path.isfile(path) else False

    return data.to_csv(path, mode="a", header=header, index=False)

In [785]:
def is_time(path: str):
    df = pd.read_csv(path)

        # NOTE: Verify that the date and time isn't already in the csv file before fetching new data and appending.

is_time("test.csv")

In [786]:
data = fetch()
data = prepare_data(data)
append_to_csv(data=data, path="test.csv")

# NOTE: Sleep? Schedule? Cron job? Some kind of event loop.

In [787]:
data = pd.read_csv("test.csv")
df = pd.DataFrame(data)
df

,time,date,air_pressure_at_sea_level,air_temperature,air_temperature_percentile_10,air_temperature_percentile_90,cloud_area_fraction,cloud_area_fraction_high,cloud_area_fraction_low,cloud_area_fraction_medium,dew_point_temperature,fog_area_fraction,relative_humidity,ultraviolet_index_clear_sky,wind_from_direction,wind_speed,wind_speed_of_gust,wind_speed_percentile_10,wind_speed_percentile_90
0,15:00:00,2021-10-27,998.7,12.1,11.4,12.9,39.4,0.0,38.6,0.9,8.2,0.0,76.6,0.0,253.0,4.2,6.9,4.0,5.5
1,15:00:00,2021-10-27,998.7,12.1,11.4,12.9,39.4,0.0,38.6,0.9,8.2,0.0,76.6,0.0,253.0,4.2,6.9,4.0,5.5
2,15:00:00,2021-10-27,998.7,12.1,11.4,12.9,39.4,0.0,38.6,0.9,8.2,0.0,76.6,0.0,253.0,4.2,6.9,4.0,5.5
3,15:00:00,2021-10-27,998.7,12.1,11.4,12.9,39.4,0.0,38.6,0.9,8.2,0.0,76.6,0.0,253.0,4.2,6.9,4.0,5.5


# PREDICTIONS

In [788]:
# Make predictions based on the data we gathered